In [163]:
import pandas as pd
from matplotlib import pyplot as plt
import json
pd.options.display.max_colwidth = 300
import config
data_root= config.DATA_ROOT
from tqdm import tqdm
tqdm.pandas()
pd.options.display.max_colwidth=5000

In [167]:
remove_cols=['Unnamed: 0', 'all_awardings', 'allow_live_comments', 
       'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
       'author_flair_type', 'author_fullname', 'author_patreon_flair',
       'author_premium',  'can_mod_post', 'contest_mode',
        'domain', 'full_link', 'gildings', 
       'is_crosspostable', 'is_meta', 'wls','removed_by_category', 'distinguished',3
       'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
       'link_flair_background_color', 'link_flair_richtext','link_flair_template_id',
       'link_flair_text_color', 'link_flair_type', 'locked', 'media',
       'media_embed', 'media_only', 'parent_whitelist_status',
        'over_18', 'pinned', 'post_hint',
       'preview', 'retrieved_on', 'secure_media',
       'secure_media_embed', 'send_replies', 'spoiler', 'stickied',
        'subreddit_subscribers', 'subreddit_type',
       'thumbnail', 'thumbnail_height', 'thumbnail_width', 
        'treatment_tags',  'url','hidden', 'quarantine', 'subreddit_name_prefixed',
       'suggested_sort','og_description', 'og_title', 'brand_safe', 'rte_mode','previous_visits','can_gild','category','content_categories','hidden','quarantine','subreddit_name_prefixed','suggested_sort','approved_at_utc','view_count',
       'url_overridden_by_dest', 'author_is_blocked', 'is_created_from_ads_ui',
       'author_flair_background_color', 'author_flair_template_id',
       'author_flair_text_color', 'poll_data', 'gallery_data', 'is_gallery', 'banned_at_utc',
       'media_metadata', 'edited','no_follow', 'pwls', 'awarders', 'whitelist_status', 'banned_by','crosspost_parent_list','crosspost_parent', 'updated_utc','author_cakeday', 'gilded','updated_utc','steward_reports', 'removed_by', 'archived','link_flair_css_class','removal_reason',
]
remove_cols2=['Unnamed: 0',  
        'author_flair_background_color', 'author_flair_css_class',
       'author_flair_richtext', 'author_flair_template_id',
        'author_flair_text_color', 'author_flair_type',
        'author_patreon_flair', 'author_premium', 
       'banned_at_utc',  'can_mod_post', 'collapsed',
       'collapsed_because_crowd_control', 'collapsed_reason',
         'edited', 'gildings',
       'is_submitter', 'link_id', 'locked', 
       'permalink', 'retrieved_on',  'send_replies', 'stickied', 'top_awarded_type',
       'total_awards_received', 'treatment_tags']

SyntaxError: invalid syntax (2355063846.py, line 7)

## Edtech Help

In [168]:
edtech_help=pd.read_csv(data_root+'/reddit-posts/edtechhelp.csv', sep=',', encoding='utf-8')
edtech_help.drop(axis=1, columns=remove_cols, inplace=True)
edtech_help_cmts=pd.read_csv(data_root+'/reddit-posts/edtechhelp-comments.csv', sep=',', encoding='utf-8')
edtech_help_cmts.drop(axis=1, columns=remove_cols2, inplace=True)
edtech_help['created_utc'] = pd.to_datetime(edtech_help.created_utc, unit='s')
edtech_help_cmts['created_utc'] = pd.to_datetime(edtech_help_cmts.created_utc, unit='s')
edtech_help_cmts['approved_at_utc'] = pd.to_datetime(edtech_help_cmts.approved_at_utc, unit='s')
print(len(edtech_help), len(edtech_help_cmts))
# df = df[df.apply(lambda tweet: tweet.text.startswith("RT")==False, axis=1 )]
# print("len(df) after removing retweets : ", len(df))

KeyError: "['wls', 'removed_by_category', 'distinguished', 'link_flair_template_id', 'parent_whitelist_status', 'hidden', 'quarantine', 'subreddit_name_prefixed', 'suggested_sort', 'og_description', 'og_title', 'brand_safe', 'rte_mode', 'previous_visits', 'can_gild', 'category', 'content_categories', 'hidden', 'quarantine', 'subreddit_name_prefixed', 'suggested_sort', 'approved_at_utc', 'view_count', 'banned_at_utc', 'edited', 'pwls', 'whitelist_status', 'banned_by', 'crosspost_parent_list', 'crosspost_parent', 'updated_utc', 'author_cakeday', 'gilded', 'updated_utc', 'steward_reports', 'removed_by', 'archived', 'link_flair_css_class', 'removal_reason'] not found in axis"

In [169]:
edtech_help[['author', 'created_utc', 'title', 'selftext']].sample(5) 

,author,created_utc,title,selftext
9,TeachMe_EdTech,1626701056,Teachers and Students can Save a Copy of OneNote Class Notebooks at the end of an academic year or if they are leaving a school/university,NaN
4,TeachMe_EdTech,1620318508,Did you know you can Create an Animated GIF in PowerPoint? This is useful way to create Stop Motion Animations of a concept for students,NaN
13,ellipsisslipsin,1603979185,Help Thread,This a place to post any questions you might have (and hopefully get some answers or suggestions!)
11,YearWithTheYeti,1604003149,I used the Google Meet Breakout Rooms Extension for chrome. It was game-changing.,NaN
16,stellaismycat,1603898635,OneNote,"I'm having an issue with one note. When I paste a worksheet and set it as background, an icon with the pdf appears at the top (or bottom) and then creates a ""text box"" that the kids can't move or type on the worksheet. Deleting the PDF icon gets rid of the worksheet entirely. Is there a work around for this? See attached picture:\n\n&amp;#x200B;\n\nhttps://preview.redd.it/wenwu9qhsuv51.png?width=815&amp;format=png&amp;auto=webp&amp;s=2d4e791c5de1630c5d12352ce3ba9111a97e0b4e\n\nHelp!"


In [170]:
edtech_help_cmts[edtech_help_cmts.apply(lambda row: isinstance(row.body,str) and 'privacy' in row.body, axis=1) ]

,all_awardings,approved_at_utc,associated_award,author,author_flair_text,author_fullname,awarders,body,comment_type,created_utc,distinguished,id,no_follow,parent_id,score,subreddit,subreddit_id
21,[],NaT,NaN,ellipsisslipsin,TechMaven,t2_4btkd04t,[],"Oh those sound like such nice options! We can't use the extension in our district because they haven't signed our privacy policy, but it sounds awesome.",NaN,2020-10-29 21:57:20,NaN,gajemvp,True,t1_gaje705,1,edtechhelp,t5_3bmcnv
33,[],NaT,NaN,trixie91,NaN,t2_ecior,[],I guess they wouldn't sign a student privacy agreement. Something like that.,NaN,2020-11-12 00:23:45,NaN,gbzlfn4,True,t1_gbzl9bx,1,edtechhelp,t5_3bmcnv


In [171]:
edtech_help_cmts[['author', 'created_utc', 'body']].sample(3)

,author,created_utc,body
15,lemonalchemyst,2020-10-29 01:07:57,"It helps to organize yourself visually and spatially. Also, it’s got to be a pain to constantly try to find the ‘right window’ and make sure not to share sensitive information. \n\nSome of my coworkers won’t buy one because the school ‘needs to do that.’ I get their point, but today you can find a really decent monitor for $100 and monitors that are a few years old for even cheaper.\n\nI don’t get it. I spend $100 on delivery some nights. But this external monitor improves my life every day"
19,lemonalchemyst,2020-10-29 21:50:15,"I keep chat up on my laptop, have one screen for what students can see, then another for what just I will see (go guardian, email, grade book, stuff)\n\nI just have to be conscious with screensharing that their school computers are much smaller than what I can see\n\n24 pt font this year"
17,ellipsisslipsin,2020-10-29 18:45:47,"Exactly! We did the same thing, and we already had a desk top so now I technically have 3 screens.\n\nI have mentioned to some people that didn't pay out of pocket that they could also use their tv as a monitor, and I've had some positive feedback on that. Especially for putting the Meet up... it makes it easier to see faces for a large class."


## Sysadmin

In [172]:
sys_df = pd.read_csv(data_root+'/reddit-posts/higheredsysadmin.csv', sep=',', encoding='utf-8')

## Other subreddits

In [173]:
dfs = []
for n in ['student', 'college', 'education', 'EngineeringStudents']:#[ 'teachers', 'professors', 'k12sysadmin']:
    df = pd.read_csv(data_root+'/reddit-posts/{}.csv'.format(n), sep=',', encoding='utf-8')
    dfs.append(df)
    print(n, len(df))
df = pd.concat(dfs)
df.drop(axis=1, columns=remove_cols, inplace=True)

df.shape

/var/folders/q5/tcs66y_j6z1bcz8ffp7_1m4r0000gq/T/ipykernel_17936/2653540110.py:3: DtypeWarning: Columns (1,2,5,7,9,10,11,12,13,18,20,21,22,23,24,26,27,28,29,30,31,32,33,34,35,39,41,46,47,52,56,59,61,62,63,64,65,66,67,68,69,70,71,72,73,76,78,79,80,81,82,83,84,85,87,88,89,90,91,92,93,94,96,99,100,101,103,105) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_root+'/reddit-posts/{}.csv'.format(n), sep=',', encoding='utf-8')


teachers 165573


/var/folders/q5/tcs66y_j6z1bcz8ffp7_1m4r0000gq/T/ipykernel_17936/2653540110.py:3: DtypeWarning: Columns (2,12,19,24,26,27,29,32,34,35,39,40,51,87,88,89,90,91,92,93,94,95,97,100,102,104) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_root+'/reddit-posts/{}.csv'.format(n), sep=',', encoding='utf-8')


professors 23666


/var/folders/q5/tcs66y_j6z1bcz8ffp7_1m4r0000gq/T/ipykernel_17936/2653540110.py:3: DtypeWarning: Columns (14,15,21,22,23,24,25,27,32,33,34,40,45,46,52,65,66,69,70,79,80,81,87,90,91,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_root+'/reddit-posts/{}.csv'.format(n), sep=',', encoding='utf-8')


k12sysadmin 12659


(201898, 17)

In [174]:
df['created_utc'] = pd.to_datetime(df.created_utc, unit='s')
df['author_created_utc']=pd.to_datetime(df.author_created_utc, unit='s')

In [196]:
def contains_keyword(text, keywords):
    if not isinstance(text, str):
        return False
    text =text.lower()
    for kw in keywords:
        if kw.lower() in text:
            return True
    return False
reddit_ed_kws = [line.split(',') for line in open('reddit-ed-kw.txt')][0]
reddit_priv_kws = [line.split(',') for line in open('reddit-priv-kw.txt')][0]

In [176]:
ed_df=df[df.progress_apply(lambda row: contains_keyword(row.title, reddit_ed_kws) or \
                        contains_keyword(row.selftext, reddit_ed_kws), axis=1)]

ed_df.shape

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 201898/201898 [00:18<00:00, 10803.62it/s]


(5994, 17)

In [197]:
priv_df=ed_df[ed_df.progress_apply(lambda row: contains_keyword(row.title, reddit_priv_kws) or \
                        contains_keyword(row.selftext, reddit_priv_kws), axis=1)]

priv_df.shape

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5994/5994 [00:00<00:00, 36160.71it/s]


(174, 17)

In [198]:
priv_df.columns

Index(['author', 'created_utc', 'id', 'is_original_content', 'link_flair_text',
       'num_comments', 'num_crossposts', 'permalink', 'score', 'selftext',
       'subreddit', 'subreddit_id', 'title', 'total_awards_received',
       'upvote_ratio', 'author_created_utc', 'author_id'],
      dtype='object')

In [206]:
# priv_df[['id', 'permalink', 'link_flair_text', 'title', 'selftext']].to_excel('priv-reddit-posts2.xlsx')

In [200]:
priv_df.num_comments.sort_values(ascending=False)

12954    271
71790    102
52151     87
8543      82
54278     76
        ... 
73364      0
73366      0
73433      0
684        0
945        0
Name: num_comments, Length: 174, dtype: int64

In [201]:
priv_df.score.sort_values(ascending=False)

68464     60
6970      33
155747    26
96242     23
5067      16
          ..
73366      1
5021       0
17503      0
11520      0
12205      0
Name: score, Length: 174, dtype: int64

In [202]:
priv_df.groupby('subreddit').selftext.count()

subreddit
Professors     45
Teachers       81
k12sysadmin    43
Name: selftext, dtype: int64

In [203]:
priv_df.link_flair_text.unique()

array(['Policy &amp; Politics', 'Student',
       'Teacher Support &amp;/or Advice', 'Resignation', 'New Teacher',
       'Student Teacher Support &amp;/or Advice',
       'RANT [Students/Parents]', 'Career &amp; Interview Advice', nan,
       'Classroom Management &amp; Strategies', 'COVID-19',
       'Bad Teacher, No Apple', 'Pedagogy &amp; Best Practices',
       'SUCCESS!', 'Moderator Announcement', 'DISCUSSION :sloth:',
       'Rants / Vents', 'Advice / Support'], dtype=object)

In [204]:
priv_df.groupby('link_flair_text').selftext.count().sort_values(ascending=False)

link_flair_text
Teacher Support &amp;/or Advice            21
COVID-19                                    5
Classroom Management &amp; Strategies       5
New Teacher                                 4
Student                                     4
Bad Teacher, No Apple                       4
Rants / Vents                               3
Advice / Support                            3
Policy &amp; Politics                       2
Resignation                                 2
Career &amp; Interview Advice               2
Student Teacher Support &amp;/or Advice     2
Pedagogy &amp; Best Practices               2
Moderator Announcement                      1
RANT [Students/Parents]                     1
DISCUSSION :sloth:                          1
SUCCESS!                                    1
Name: selftext, dtype: int64